In [1]:
import numpy as np
import csv
from sklearn import linear_model, datasets, tree
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
X = np.empty((0,214), float)
Y = np.empty(0, str)
i = 0
r = csv.reader(open('Train_Data_New/Normalizedfeatures.csv',"rb"),delimiter=',') # Here your csv file
for l in r:
    if i == 1:
        X = np.append(X, np.array([l[0:214]]), axis=0)
        Y = np.append(Y, np.array([l[214]]), axis=0)
    else:
        i = 1
        features = l[0:214]
        output = l[214]
        
X = X.astype(float)
Y = Y.astype(str)

In [3]:
number_of_samples = len(Y)
#Splitting into training, validation and test sets
random_indices = np.random.permutation(number_of_samples)
#Training set
num_training_samples = int(number_of_samples*0.80)
X_train = X[random_indices[:num_training_samples]]
Y_train = Y[random_indices[:num_training_samples]]
#Test set
num_test_samples = int(number_of_samples*0.20)
X_test = X[random_indices[-num_test_samples:]]
Y_test = Y[random_indices[-num_test_samples:]]

In [5]:
# dot_data = tree.export_graphviz(model, out_file=None)
# graph = pydotplus.graph_from_dot_data(dot_data)
# graph.write_pdf("iris.pdf")

In [7]:
#Reduce the dimensionality of the feature space
# pca = PCA(svd_solver='randomized', whiten=True).fit(X)
# var_list = pca.explained_variance_ratio_.cumsum()
# for i in range(len(var_list)):
#     if var_list[i] > 0.99:
#         n_components = i+1
#         break
# print('n_components=%d'%n_components)
#Finding the top n_components principal components in the data
test_error=[]
train_error=[]
for i in range(100):
    number_of_samples = len(Y)
    #Splitting into training, validation and test sets
    random_indices = np.random.permutation(number_of_samples)
    #Training set
    num_training_samples = int(number_of_samples*0.80)
    X_train = X[random_indices[:num_training_samples]]
    Y_train = Y[random_indices[:num_training_samples]]
    #Test set
    num_test_samples = int(number_of_samples*0.20)
    X_test = X[random_indices[-num_test_samples:]]
    Y_test = Y[random_indices[-num_test_samples:]]

    pca = LinearDiscriminantAnalysis().fit(X_train,Y_train) #Used LDA for dimensionality reduction
    
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)

    model = tree.DecisionTreeClassifier(min_impurity_split=1e-2)
    model.fit(X_train_pca,Y_train)

    test_set_predictions = np.array([model.predict(X_test_pca[i].reshape((1,len(X_test_pca[0]))))[0] for i in range(X_test_pca.shape[0])])
    train_set_predictions = np.array([model.predict(X_train_pca[i].reshape((1,len(X_train_pca[0]))))[0] for i in range(X_train_pca.shape[0])])

    test_misclassification_percentage = 0.0
    for i in range(len(test_set_predictions)):
        if test_set_predictions[i]!=Y_test[i]:
            test_misclassification_percentage+=1.0
    test_misclassification_percentage *= 100.0/len(Y_test)
    train_misclassification_percentage = 0.0
    for i in range(len(train_set_predictions)):
        if train_set_predictions[i]!=Y_train[i]:
            train_misclassification_percentage+=1.0
    train_misclassification_percentage *= 100.0/len(Y_train)
    test_error.append(test_misclassification_percentage)
    train_error.append(train_misclassification_percentage)

print 'test misclassification percentage =', np.mean(test_error), '%'
print 'train misclassification percentage =', np.mean(train_error), '%'

test misclassification percentage = 63.630952381 %
train misclassification percentage = 0.0 %
